Посмотрим на результаты.

In [2]:
import torch
from torch import nn
import torch.utils.data as data_utils
import torchvision as tv
from torchsummary import summary
import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


> Лучший результат получен на самой простой архитектуре, за счет того, что благодаря ее высокому быстродействию, я не ограничивал размер тренировочного датасета. Следовательно можно сделать вывод, что для глубоких сверточных сетей количество обучающих примеров и общее время обучения (больше, чем количество эпох) являются определяющими для качества обучения параметрами.

> В целом, усложнение архитектуры ведет к увеличению качества обучения при одинаковом времени обучения и одинаковых тренировочных данных, но время обучения (обработки эквивалентного количества примеров) возрастает.

> Казалось бы, на сбалансированном датасете результаты обучения должны быть выше, чем на несбалансированном. Полученнный результат требуованияет исследования. Возможно 2400 примеров для каждого класса - это маловато для хорошего результата, а возможно для сбалансированного датасета были отобраны более неоднозначные примеры.

> Анализ ниже показал, что хотя я не пользовался всякими преремешиваниями, классы во всех выборках присутствуют в полном объеме.

> В классе датасета pytorch нашлись только очень скромные инструменты для количественно/качественного анализа, особенно для subset набора, который ссылается обратно на базовый датасет. Не удалось найти автоматизированного способа перегнать данные из датасета в pandas, а делать это методом перебора кажется кривовато и долговато.  
Не знаете ли случайно более красивого метода?






  


|Архитектура|баланс ds|количество|эпох|train loss|train acc|test acc|time/epoch|
|:------------------------|:---|:---|:---|:-------|:--------|:---|:---|
|ResNet18|no|5| 697 932|0.6565|0.798|0.798|1943.0|
|ResNet18|yes|5|112 800|1.1162|0.733|0.574|484.4|
|VGG16|no|5|112 800|0.6622|0.798|0.795|1388.3|
|VGG16|yes|5|112 800|0.6284|0.812|0.626|1355.5|
|Inception v3|no|5|112 800|2.0974|0.573|0.617|1228.2|
|Inception v3|yes|5|112 800|2.1867|0.559|0.488|1206.3|
|DenseNet161|no|5|112 800|1.0813|0.725|0.729|1768.3|
|DenseNet161|yes|5|112 800|1.0768|0.738|0.579|1770.6|

In [8]:
#BATCH_SIZE = 256
BATCH_SIZE = 1

indices = torch.arange(112800)

transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3), # предобученные модели работают с 3х канальными рисунками, поэтому искусственно делаем из одного три канала
    tv.transforms.Resize((224, 224)), # трансформируем наши изображения до требуемого размера
    tv.transforms.ToTensor()
])
train_dataset_bc = tv.datasets.EMNIST(root="./data", split='byclass', train=True, transform=transoforms, download=True)
test_dataset_bc = tv.datasets.EMNIST(root="./data", split='byclass', train=False, transform=transoforms, download=True)

train_dataset_bc_min = data_utils.Subset(train_dataset_bc, indices)
test_dataset_bc_min = data_utils.Subset(test_dataset_bc, indices)
train_iter_bc = torch.utils.data.DataLoader(train_dataset_bc_min, batch_size=BATCH_SIZE)
test_iter_bc = torch.utils.data.DataLoader(test_dataset_bc_min, batch_size=BATCH_SIZE)
# уменьшаем выборку данных, чтобы протестировать код на cpu быстрее
# 112800 ограничили количеством в сбалансированном датасете

#train_iter_bc = torch.utils.data.DataLoader(train_dataset_bc, batch_size=BATCH_SIZE)
#test_iter_bc = torch.utils.data.DataLoader(test_dataset_bc, batch_size=BATCH_SIZE)

#############################################3

train_dataset_bl = tv.datasets.EMNIST(root="./data", split='balanced', train=True, transform=transoforms, download=True)
test_dataset_bl = tv.datasets.EMNIST(root="./data", split='balanced', train=False, transform=transoforms, download=True)

#train_dataset_bl_min = data_utils.Subset(train_dataset_bl, indices)
#test_dataset_bl_min = data_utils.Subset(test_dataset_bl, indices)
#train_iter_bl = torch.utils.data.DataLoader(train_dataset_bl_min, batch_size=BATCH_SIZE)
#test_iter_bl = torch.utils.data.DataLoader(test_dataset_bl_min, batch_size=BATCH_SIZE)
# уменьшаем выборку данных, чтобы протестировать код на cpu быстрее


train_iter_bl = torch.utils.data.DataLoader(train_dataset_bl, batch_size=BATCH_SIZE)
test_iter_bl = torch.utils.data.DataLoader(test_dataset_bl, batch_size=BATCH_SIZE)

In [9]:
uniq_train_bc = train_dataset_bc.targets.unique(return_counts=True)
uniq_train_bc

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60, 61]),
 tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,
          6407,  3878, 10094,  4562,  4934,  9182,  2517,  3152, 11946,  3762,
          2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820,
         12602,  4637,  4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177,
         24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,  2645, 11418,
          2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,
          2365,  2725]))

In [22]:
targets_train_iter_bc = {}
for X,y in train_iter_bc:
  yyy = y.item()
  if yyy in targets_train_iter_bc:
    targets_train_iter_bc[yyy] += 1  # Увеличиваем значение для существующего элемента
  else:
    targets_train_iter_bc[yyy] = 1  # Добавляем новый элемент в словарь с начальным значением 1


Для тренировочных данных bc

In [23]:
keys = targets_train_iter_bc.keys()
for k in sorted(targets_train_iter_bc.keys()):
  print(f'k: {k} v: {targets_train_iter_bc[k]}')

k: 0 v: 5601
k: 1 v: 6175
k: 2 v: 5510
k: 3 v: 5631
k: 4 v: 5566
k: 5 v: 5062
k: 6 v: 5546
k: 7 v: 5756
k: 8 v: 5468
k: 9 v: 5410
k: 10 v: 1035
k: 11 v: 624
k: 12 v: 1663
k: 13 v: 760
k: 14 v: 779
k: 15 v: 1524
k: 16 v: 397
k: 17 v: 499
k: 18 v: 1917
k: 19 v: 573
k: 20 v: 388
k: 21 v: 791
k: 22 v: 1456
k: 23 v: 1391
k: 24 v: 4123
k: 25 v: 1335
k: 26 v: 438
k: 27 v: 837
k: 28 v: 3380
k: 29 v: 1522
k: 30 v: 1998
k: 31 v: 763
k: 32 v: 781
k: 33 v: 465
k: 34 v: 765
k: 35 v: 428
k: 36 v: 1563
k: 37 v: 799
k: 38 v: 428
k: 39 v: 1676
k: 40 v: 3947
k: 41 v: 398
k: 42 v: 599
k: 43 v: 1450
k: 44 v: 467
k: 45 v: 278
k: 46 v: 379
k: 47 v: 2509
k: 48 v: 434
k: 49 v: 1756
k: 50 v: 427
k: 51 v: 402
k: 52 v: 464
k: 53 v: 2290
k: 54 v: 463
k: 55 v: 2986
k: 56 v: 480
k: 57 v: 484
k: 58 v: 448
k: 59 v: 453
k: 60 v: 383
k: 61 v: 480


In [24]:
targets_test_iter_bc = {}
for X,y in test_iter_bc:
  yyy = y.item()
  if yyy in targets_test_iter_bc:
    targets_test_iter_bc[yyy] += 1  # Увеличиваем значение для существующего элемента
  else:
    targets_test_iter_bc[yyy] = 1  # Добавляем новый элемент в словарь с начальным значением 1


Для тестовых данных bc

In [25]:
keys = targets_test_iter_bc.keys()
for k in sorted(targets_test_iter_bc.keys()):
  print(f'k: {k} v: {targets_test_iter_bc[k]}')

k: 0 v: 5615
k: 1 v: 6143
k: 2 v: 5692
k: 3 v: 5785
k: 4 v: 5448
k: 5 v: 5038
k: 6 v: 5548
k: 7 v: 5971
k: 8 v: 5474
k: 9 v: 5494
k: 10 v: 1028
k: 11 v: 635
k: 12 v: 1683
k: 13 v: 747
k: 14 v: 826
k: 15 v: 1406
k: 16 v: 438
k: 17 v: 507
k: 18 v: 1979
k: 19 v: 610
k: 20 v: 364
k: 21 v: 790
k: 22 v: 1440
k: 23 v: 1312
k: 24 v: 4026
k: 25 v: 1367
k: 26 v: 403
k: 27 v: 792
k: 28 v: 3380
k: 29 v: 1533
k: 30 v: 1931
k: 31 v: 773
k: 32 v: 774
k: 33 v: 418
k: 34 v: 766
k: 35 v: 452
k: 36 v: 1593
k: 37 v: 817
k: 38 v: 417
k: 39 v: 1635
k: 40 v: 3974
k: 41 v: 387
k: 42 v: 576
k: 43 v: 1425
k: 44 v: 410
k: 45 v: 309
k: 46 v: 454
k: 47 v: 2457
k: 48 v: 450
k: 49 v: 1824
k: 50 v: 452
k: 51 v: 359
k: 52 v: 489
k: 53 v: 2242
k: 54 v: 426
k: 55 v: 2879
k: 56 v: 466
k: 57 v: 455
k: 58 v: 457
k: 59 v: 452
k: 60 v: 369
k: 61 v: 438


In [26]:
targets_train_iter_bl = {}
for X,y in train_iter_bl:
  yyy = y.item()
  if yyy in targets_train_iter_bl:
    targets_train_iter_bl[yyy] += 1  # Увеличиваем значение для существующего элемента
  else:
    targets_train_iter_bl[yyy] = 1  # Добавляем новый элемент в словарь с начальным значением 1


Для тренировочных данных bl

In [27]:
keys = targets_train_iter_bl.keys()
for k in sorted(targets_train_iter_bl.keys()):
  print(f'k: {k} v: {targets_train_iter_bl[k]}')

k: 0 v: 2400
k: 1 v: 2400
k: 2 v: 2400
k: 3 v: 2400
k: 4 v: 2400
k: 5 v: 2400
k: 6 v: 2400
k: 7 v: 2400
k: 8 v: 2400
k: 9 v: 2400
k: 10 v: 2400
k: 11 v: 2400
k: 12 v: 2400
k: 13 v: 2400
k: 14 v: 2400
k: 15 v: 2400
k: 16 v: 2400
k: 17 v: 2400
k: 18 v: 2400
k: 19 v: 2400
k: 20 v: 2400
k: 21 v: 2400
k: 22 v: 2400
k: 23 v: 2400
k: 24 v: 2400
k: 25 v: 2400
k: 26 v: 2400
k: 27 v: 2400
k: 28 v: 2400
k: 29 v: 2400
k: 30 v: 2400
k: 31 v: 2400
k: 32 v: 2400
k: 33 v: 2400
k: 34 v: 2400
k: 35 v: 2400
k: 36 v: 2400
k: 37 v: 2400
k: 38 v: 2400
k: 39 v: 2400
k: 40 v: 2400
k: 41 v: 2400
k: 42 v: 2400
k: 43 v: 2400
k: 44 v: 2400
k: 45 v: 2400
k: 46 v: 2400


In [28]:
targets_test_iter_bl = {}
for X,y in test_iter_bl:
  yyy = y.item()
  if yyy in targets_test_iter_bl:
    targets_test_iter_bl[yyy] += 1  # Увеличиваем значение для существующего элемента
  else:
    targets_test_iter_bl[yyy] = 1  # Добавляем новый элемент в словарь с начальным значением 1


Для тестовых данных bl

In [29]:
keys = targets_test_iter_bl.keys()
for k in sorted(targets_test_iter_bl.keys()):
  print(f'k: {k} v: {targets_test_iter_bl[k]}')

k: 0 v: 400
k: 1 v: 400
k: 2 v: 400
k: 3 v: 400
k: 4 v: 400
k: 5 v: 400
k: 6 v: 400
k: 7 v: 400
k: 8 v: 400
k: 9 v: 400
k: 10 v: 400
k: 11 v: 400
k: 12 v: 400
k: 13 v: 400
k: 14 v: 400
k: 15 v: 400
k: 16 v: 400
k: 17 v: 400
k: 18 v: 400
k: 19 v: 400
k: 20 v: 400
k: 21 v: 400
k: 22 v: 400
k: 23 v: 400
k: 24 v: 400
k: 25 v: 400
k: 26 v: 400
k: 27 v: 400
k: 28 v: 400
k: 29 v: 400
k: 30 v: 400
k: 31 v: 400
k: 32 v: 400
k: 33 v: 400
k: 34 v: 400
k: 35 v: 400
k: 36 v: 400
k: 37 v: 400
k: 38 v: 400
k: 39 v: 400
k: 40 v: 400
k: 41 v: 400
k: 42 v: 400
k: 43 v: 400
k: 44 v: 400
k: 45 v: 400
k: 46 v: 400
